# Composing Music with xLSTM.

This notebook shows you how to compose music with xLSTM

On top of the installation instructions from the README, you might want to install the following packages:

In [ ]:
# Please uncomments. This is for the first time setup
#!pip install note-seq
#!pip install --upgrade bokeh==2.4.3

## Load the language model.

We will use this model: https://huggingface.co/TristanBehrens/jsfakes-music-xlstm/tree/main

In [1]:
import sys
sys.path.append("..")
from source.languagemodel import LanguageModel
import note_seq

# Select a model to load.
#model_path_or_repo = "TristanBehrens/jsfakes-music-transformer"
#model_path_or_repo = "TristanBehrens/jsfakes-music-pharia"
#model_path_or_repo = "TristanBehrens/jsfakes-music-mamba"
#model_path_or_repo = "TristanBehrens/jsfakes-music-xlstm"
#model_path_or_repo = "TristanBehrens/bach-garland-transformer"
model_path_or_repo = "TristanBehrens/bach-garland-pharia"
#model_path_or_repo = "TristanBehrens/bach-garland-mamba"
#model_path_or_repo = "TristanBehrens/bach-garland-xlstm"
#model_path_or_repo = "TristanBehrens/bach-garland-mambaplus"


# Load the model.
model = LanguageModel(
    model_path_or_repo,
    config_overrides={"context_length": 16_384},
)
model.summary()

/home/hordak/miniconda3/envs/helibrunna/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   ▄█    █▄       ▄████████  ▄█        ▄█  ▀█████████▄     ▄████████ ███    █▄  ███▄▄▄▄   ███▄▄▄▄      ▄████████ 
  ███    ███     ███    ███ ███       ███    ███    ███   ███    ███ ███    ███ ███▀▀▀██▄ ███▀▀▀██▄   ███    ███ 
  ███    ███     ███    █▀  ███       ███▌   ███    ███   ███    ███ ███    ███ ███   ███ ███   ███   ███    ███ 
 ▄███▄▄▄▄███▄▄  ▄███▄▄▄     ███       ███▌  ▄███▄▄▄██▀   ▄███▄▄▄▄██▀ ███    ███ ███   ███ ███   ███   ███    ███ 
▀▀███▀▀▀▀███▀  ▀▀███▀▀▀     ███       ███▌ ▀▀███▀▀▀██▄  ▀▀███▀▀▀▀▀   ███    ███ ███   ███ ███   ███ ▀███████████ 
  ███    ███     ███    █▄  ███       ███    ███    ██▄ ▀███████████ ███    ███ ███   ███ ███   ███   ███    ███ 
  ███    ███     ███    ███ ███▌    ▄ ███    ███    ███   ███    ███ ███    ███ ███   ███ ███   ███   ███    ███ 
  ███    █▀      ██████████ █████▄▄██ █▀   ▄█████████▀    ███    ███ ████████▀   ▀█   █▀   ▀█   █▀    ███    █▀  
                            ▀                             ███    ███                    

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 2259.05it/s]


{
    "type": "pharia",
    "attention_bias": true,
    "attention_dropout": 0.0,
    "eos_token_id": 0,
    "bos_token_id": 127179,
    "pad_token_id": 1,
    "hidden_act": "gelu",
    "hidden_size": 64,
    "initializer_range": 0.02,
    "intermediate_size": 128,
    "max_position_embeddings": 2048,
    "mlp_bias": true,
    "num_attention_heads": 4,
    "num_hidden_layers": 4,
    "num_key_value_heads": 4,
    "rope_scaling": null,
    "rope_theta": 1000000,
    "tie_word_embeddings": false,
    "use_cache": true,
    "context_length": 16384,
    "vocab_size": 178
}
178 64 1
PhariaModel(
  (embed_tokens): Embedding(178, 64, padding_idx=1)
  (layers): ModuleList(
    (0-3): 4 x PhariaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=64, out_features=64, bias=True)
        (k_proj): Linear(in_features=64, out_features=64, bias=True)
        (v_proj): Linear(in_features=64, out_features=64, bias=True)
        (o_proj): Linear(in_features=64, out_feat

/home/hordak/miniconda3/envs/helibrunna/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2165: FutureWarning: Calling PreTrainedTokenizerFast.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
/home/hordak/miniconda3/envs/helibrunna/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Utility code for music

Here is some code that will map anything that the model generates into a music representation that is close to GM MIDI.

In [2]:
import copy

def handle_garland_tokens(tokens):
    song_data = {}

    song_data["tracks"] = []

    current_track_index = 0
    current_timestep = 0
    for token in tokens:
        if token == "GARLAND_START":
            pass
        elif token == "BAR_START":
            if current_track_index == len(song_data["tracks"]):
                song_data["tracks"] += [{"bars": [], "instrument": "0"}]
            bar_data = {"notes": []}
            song_data["tracks"][current_track_index]["bars"] += [bar_data]
            current_timestep = 0
        elif token.startswith("INST="):
            instrument = token.split("=")[1]
            song_data["tracks"][current_track_index]["instrument"] = instrument
        elif token.startswith("DENSITY="):
            pass
        elif token.startswith("NOTE_ON="):
            note_pitch = int(token.split("=")[1])
            note_data = {
                "note": note_pitch,
                "start": current_timestep,
                "end": current_timestep,
                "veloctiy": 80
            }
            song_data["tracks"][current_track_index]["bars"][-1]["notes"] += [note_data]
            pass
        elif token.startswith("TIME_DELTA="):
            current_timestep += int(token.split("=")[1])
        elif token.startswith("NOTE_OFF="):
            note_pitch = int(token.split("=")[1])
            for note_data in song_data["tracks"][current_track_index]["bars"][-1]["notes"]:
                if note_data["note"] == note_pitch and note_data["start"] == note_data["end"]:
                    note_data["end"] = current_timestep
                    break
            pass
        elif token == "BAR_END":
            current_track_index += 1
        elif token == "NEXT":
            current_track_index = 0
        elif token == "GARLAND_END":
            pass
        elif token == "[PAD]":
            pass
        elif token == "[EOS]":
            pass
        else:
            raise Exception(f"Unknown token: {token}")
    
    assert isinstance(song_data, dict)
    return song_data

def convert_songdata_to_notesequence(song_data:dict, quantize_steps_per_quarter=8, remove_disabled_tracks=True):

    assert isinstance(song_data, dict), f"Invalid song data type: {type(song_data)}"

    # Clone the song data.
    song_data = copy.deepcopy(song_data)

    # Sort the tracks by instrument.
    assert "tracks" in song_data, f"Invalid song data: {song_data.keys()}"
    tracks = sorted(song_data["tracks"], key=lambda t: t["instrument"])
    song_data["tracks"] = tracks

    # Remove tracks that are not enabled.
    if remove_disabled_tracks:
        song_data["tracks"] = [t for t in song_data["tracks"] if t.get("enabled", True)]

    # Create an empy note sequence.
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()

    # Add the tempo.
    bpm = song_data["bpm"] if "bpm" in song_data else 120
    note_sequence.tempos.add().qpm = bpm

    # Compute some lengths.
    step_length_seconds = 60.0 / bpm / quantize_steps_per_quarter
    bar_length_seconds = 4 * step_length_seconds * quantize_steps_per_quarter

    # Get the instruments.
    instruments = list(set([t["instrument"] for t in song_data["tracks"]]))

    # Add the tracks.
    for track_index, track_data in enumerate(song_data["tracks"]):
        instrument = track_data["instrument"]
        for bar_index, bar_data in enumerate(track_data["bars"]):
            bar_start_time = bar_index * bar_length_seconds
            for note_data in bar_data["notes"]:
                assert "note" in note_data
                assert "start" in note_data
                assert "end" in note_data
                note = note_sequence.notes.add()
                #note.instrument = instrument TODO
                note.pitch = note_data["note"]
                note.start_time = note_data["start"] * step_length_seconds + bar_start_time
                note.end_time = note_data["end"] * step_length_seconds + bar_start_time
                if "velocity" in note_data:
                    note.velocity = note_data["velocity"]
                else:
                    note.velocity = 80
                note.instrument = track_index
                if instrument == "drums":
                    note.is_drum = True
                else:
                    note.is_drum = False
                    note.program = int(instrument)

    return note_sequence

## Generate music

In [3]:
# The temperature of the generation. The higher the temperature, the more random the output.
temperature = 0.5

# The maximum length of the generated music.
max_length = 16_384

# When to stop the generation.
end_tokens = ["NEXT"]

# Compose the music iterativelybar by bar.
output = "GARLAND_START"
for iteration in range(10):
    output_dict = model.generate(
        prompt=output,
        temperature=temperature,
        max_length=max_length,
        end_tokens=end_tokens,
        forbidden_tokens=["[PAD]", "[EOS]", "GARLAND_END"],
        return_structured_output=True
    )
    for key, value in output_dict.items():
        print(key, value)
    output = output_dict["output"]
    length = len(output.split(" "))
    print(f"Iteration {iteration}, length {length:_}")
    if length >= max_length:
        break

output GARLAND_START BAR_START DENSITY=3 NOTE_ON=65 TIME_DELTA=8 NOTE_OFF=65 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 BAR_END BAR_START DENSITY=3 NOTE_ON=53 TIME_DELTA=8 NOTE_OFF=53 NOTE_ON=52 TIME_DELTA=8 NOTE_OFF=52 NOTE_ON=48 TIME_DELTA=8 NOTE_OFF=48 NOTE_ON=48 TIME_DELTA=8 NOTE_OFF=48 BAR_END BAR_START DENSITY=3 NOTE_ON=60 TIME_DELTA=8 NOTE_OFF=60 NOTE_ON=60 TIME_DELTA=8 NOTE_OFF=60 NOTE_ON=60 TIME_DELTA=8 NOTE_OFF=60 NOTE_ON=60 TIME_DELTA=8 NOTE_OFF=60 BAR_END BAR_START DENSITY=3 NOTE_ON=69 TIME_DELTA=8 NOTE_OFF=69 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 NOTE_ON=72 TIME_DELTA=8 NOTE_OFF=72 NOTE_ON=72 TIME_DELTA=8 NOTE_OFF=72 BAR_END NEXT
elapsed_time 0.5048372745513916
tokens_per_second 120.83101441787517
Iteration 0, length 62
output GARLAND_START BAR_START DENSITY=3 NOTE_ON=65 TIME_DELTA=8 NOTE_OFF=65 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=67 BAR_END B

In [4]:
tokens = output.split()
song_data = handle_garland_tokens(tokens)
note_sequence = convert_songdata_to_notesequence(song_data)

note_seq.plot_sequence(note_sequence)
note_seq.play_sequence(note_sequence)

Loading BokehJS ...

# Thank you!